In [1]:
import cv2
import numpy as np
import time
from autoplay import pyo
import context


WxPython is not found for the current python version.
Pyo will use a minimal GUI toolkit written with Tkinter (if available).
This toolkit has limited functionnalities and is no more
maintained or updated. If you want to use all of pyo's
GUI features, you should install WxPython, available here:
http://www.wxpython.org/



In [2]:
## effect description
effects = {
    'disortion':{
        'x':10,
        'y':50,
        'w':140,
        'h':140,
        'isExpressive':False,
        'effectName':"disortion",
        'color':(255,255,0),
        'state':0
        
    },
    'delay':{
        'x':170,
        'y':50,
        'w':140,
        'h':140,
        'isExpressive':True,
        'effectName':"delay",
        'color':(255,0,255),
        'state':0
    }
}


## init for previous points

prev_x , prev_y = (0,0)

## pyo Object placeholders:

pyoEffects ={
    'delay':None,
    'disortion':None,
    'delay_prev':0,
    'feedback_prev':0,
    'reverb':None
}
screen_w = 340
screen_h = 220


In [3]:
def nothing(x):
    pass

In [4]:
def processBoard(img,input_x, input_y):
    for effectName, effect in effects.items():
        cv2.rectangle(img,(effect['x'],effect['y']),
                      (effect['x']+effect['w'],effect['y']+effect['h']),effect['color'], 3)
        processInput(effectName,effect,input_x,input_y)
    
        
    

In [5]:
def checkInBoundaries(x,y,effect):
    if  x > effect['x'] and x < effect['x'] + effect['w'] and y > effect['y'] and  y > effect['y'] and y < effect['y']+effect['h']:
        bInBoundaries = True
    else:
        bInBoundaries = False
    return bInBoundaries

In [6]:
def processInput(effectName, effect,x,y):
    #print(f'x coordinate is {x} , y coordinate is {y}')
    
    ## if the current point is in effect boundaries
    if checkInBoundaries(x,y,effect):    
        # check if the point was in there the last frame
        #don't switch states, check if effect is expressive 
        if checkInBoundaries(prev_x,prev_y,effect):
            if effect['isExpressive']:
            ## change the effect expressive values instead
            ##TODO
                changeExpressive(effect,x,y)
            else:
            ## do nothing, still in the boundaries
                pass
        else:
            #increment the state
            effect['state']= (effect['state'] +1) % 4
            processEffect(effect)
            print(f'effect {effectName} is in state {effect["state"]} ')
    else:
        ## check if the point was previously in 
        if checkInBoundaries(prev_x,prev_y,effect):
            #increment the state
            effect['state']= (effect['state'] +1) % 4
            processEffect(effect)
            print(f'effect {effectName} is in state {effect["state"]} ')
        else:
        ## nothing changed
            pass

In [7]:
def changeExpressive(effect,input_x,input_y):
    if effect['isExpressive']:
        if effect['effectName'] == 'disortion':
            x_adjusted = (input_x -effect['x'])/effect['w']
            y_adjusted = (input_y -effect['y'])/effect['h']
            #print(f'adjusted x coordinate is {x_adjusted} , y coordinate is {y_adjusted}')
            pyoEffects['disortion'].setDrive(x_adjusted)
            pyoEffects['disortion'].setSlope(y_adjusted)
        if effect['effectName'] == 'delay':
            x_adjusted = (input_x -effect['x'])/effect['w']
            y_adjusted = (input_y -effect['y'])/effect['h']
           # print(f'adjusted x coordinate is {x_adjusted} , y coordinate is {y_adjusted}')
            
            ## if the step is bigger than 0.1 change the dellay counter
            if abs(pyoEffects['delay_prev']-x_adjusted)>0.1:
                pyoEffects['delay'].setDelay(x_adjusted)
                pyoEffects['delay_prev']=x_adjusted
            if abs(pyoEffects['feedback_prev']-y_adjusted)>0.1:
                pyoEffects['delay'].setDelay(y_adjusted)
                pyoEffects['feedback_prev']=y_adjusted
            #pyoEffects['delay'].setFeedback(x_adjusted)


In [8]:
def processEffect(effect,input_x=False,input_y=False):
    if effect['effectName'] == 'disortion':
        print('processing disortion')
        if effect['state'] == 0 or effect['state'] ==3:
            ## turn off the effect
            pyoEffects['disortion'].stop()
        else:

            pyoEffects['disortion'] = pyo.Disto(a, drive=0.91, slope=.8, mul=.15).out()
    elif effect['effectName'] == 'delay':
        print('processing delay')
        if effect['state'] == 0 or effect['state'] ==3:
            ## turn off the effect
            pyoEffects['delay'].stop()
        elif effect['state'] == 1:
            lfo_delay = pyo.Sine(freq=4, phase=.5, mul=.5, add=.3)
            pyoEffects['delay'] = pyo.Delay(a, delay=.5, feedback=lfo_delay, maxdelay=3).out()
        

In [ ]:
# cv2.namedWindow("Tracking")
# cv2.createTrackbar("LH","Tracking",76,255,nothing)
# cv2.createTrackbar("LS","Tracking",16,255,nothing)
# cv2.createTrackbar("LV","Tracking",100,255,nothing)
# cv2.createTrackbar("UH","Tracking",76,255,nothing)
# cv2.createTrackbar("US","Tracking",98,255,nothing)
# cv2.createTrackbar("UV","Tracking",100,255,nothing)

kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))


cap = cv2.VideoCapture(1)

count = 0
background=0

for i in range(60):
    ret,background = cap.read()

    
## start the pyo Server

s = pyo.Server(audio='coreaudio', nchnls=2).boot()
s.start()
a = pyo.Input(chnl=0)
## PLAIN IS REVERB
pyoEffects['reverb'] = pyo.Freeverb(a).out()
## output blank audio
#a.out() #huh?
while(cap.isOpened()):
    ret, img = cap.read()
    if not ret:
        break
    count+=1
    img=cv2.resize(img,(screen_w,screen_h))
    img = cv2.flip(img,1)
    blured_frame = cv2.GaussianBlur(img,(5,5),0)
#     thres = cv2.threshold(blured_frame, 60, 255, cv2.THRESH_BINARY)[1]
    
    hsv = cv2.cvtColor(blured_frame, cv2.COLOR_BGR2HSV)

    
#     l_h = cv2.getTrackbarPos("LH","Tracking")
#     l_s = cv2.getTrackbarPos("LS","Tracking")
#     l_v = cv2.getTrackbarPos("LV","Tracking")
#     u_h = cv2.getTrackbarPos("UH","Tracking")
#     u_s = cv2.getTrackbarPos("US","Tracking")
#     u_v = cv2.getTrackbarPos("UV","Tracking")

    ##
    lower = np.array([0, 0, 226])
    upper = np.array([51, 180,255])
#     lower = np.array([l_h, l_h, l_v])
#     upper = np.array([u_h, u_h,u_v])    
    mask = cv2.inRange(hsv,lower,upper)

    maskOpen=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
    maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

    maskFinal=maskClose
    conts,h=cv2.findContours(maskFinal.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    #cv2.drawContours(img,conts,-1,(255,0,0),3)
    
    input_x,input_y = (0,0)
    area = 0
    for i in range(len(conts)):
        x,y,w,h=cv2.boundingRect(conts[i])
        
        ## get the countour with highest area value ( TODO be more efficient)
        if w*h > area:
            area = w*h
            input_x,input_y = (x,y)
        cv2.circle(img,(x,y), 5, (255,0,0), -1)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255), 2)
    
    ## Draw the input circle now
    cv2.circle(img,(input_x,input_y), 10, (255,128,128), -1)
    processBoard(img,input_x,input_y)
    
    # keep current x,y pos for next frame
    prev_x , prev_y = (input_x,input_y)
    
#     cv2.imshow("maskClose",maskClose)
#     cv2.imshow("maskOpen",maskOpen)
#     cv2.imshow("mask",mask)
    cv2.imshow("ARBoard",img)

    k = cv2.waitKey(10)
    if k == 27:
        break

Pyo warning: Portmidi warning: no midi device found!
Portmidi closed.
processing delay
effect delay is in state 1 
processing disortion
effect disortion is in state 1 
processing delay
effect delay is in state 2 
processing disortion
effect disortion is in state 2 
processing delay
effect delay is in state 3 
processing disortion
effect disortion is in state 3 
processing delay
effect delay is in state 0 
processing disortion
effect disortion is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing disortion
effect disortion is in state 1 
processing delay
effect delay is in state 2 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing delay
effect delay is in state 3 
processing 

processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing

processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in sta

processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing delay
effect delay is in state 1 
processing disortion
effect disortion is in state 1 
processing delay
effect delay is in state 2 
processing disortion
effect disortion is in state 2 
processing delay
effect delay is in state 3 
processing disortion
effect disortion is in state 3 
processing delay
effect delay is in state 0 
processing d

processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay


processing disortion
effect disortion is in state 2 
processing delay
effect delay is in state 3 
processing disortion
effect disortion is in state 3 
processing delay
effect delay is in state 0 
processing disortion
effect disortion is in state 0 
processing delay
effect delay is in state 1 
processing disortion
effect disortion is in state 1 
processing delay
effect delay is in state 2 
processing disortion
effect disortion is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing dela

processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing delay
effect delay is in state 3 
processing delay
effect delay is in state 0 
processing delay
effect delay is in state 1 
processing delay
effect delay is in state 2 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in state 0 
processing disortion
effect disortion is in state 1 
processing disortion
effect disortion is in state 2 
processing disortion
effect disortion is in state 3 
processing disortion
effect disortion is in 